# 12 - Garden Groups

https://adventofcode.com/2024/day/12


In [277]:
// Imports

import colors from "../../utils/colors.ts"
import objects from "../../utils/objects.ts"
import strings from "../../utils/strings.ts"
import numbers from "../../utils/numbers.ts"
import plots from "../../utils/plots.ts";

In [278]:
// Read Input

class plot {
    plantType: string;
    region: string;
    x: number;
    y: number

    constructor(plantType: string, region: string, x: number, y: number) {
        this.plantType = plantType;
        this.region = region;
        this.x = x,
            this.y = y
    }
}

const file = await Deno.readTextFile("input-base.txt");
const area = file.split("\n").map((line, i) => line.split('').map((el, j) => new plot(el, null, i, j)));
[area.length, area[0].length]

[ 10, 10 ]

In [279]:
// Prepare Data
const populateRegions = (area: plot[][]) => {
    let currentRegion = 0;
    const regions = {};
    for (let i = 0; i < area.length; i++) {
        for (let j = 0; j < area[i].length; j++) {
            const current = area[i][j];
            const left = j > 0 ? area[i][j - 1] : null;
            const top = i > 0 ? area[i - 1][j] : null;
            if (left &&
                top &&
                left.plantType === current.plantType &&
                top.plantType === current.plantType &&
                left.region !== top.region) {
                const leftRegion = left.region;
                const topRegion = top.region;

                current.region = topRegion;

                regions[leftRegion].forEach(el => {
                    el.region = topRegion;
                    area[el.x][el.y].region = topRegion;
                    regions[topRegion].push(el);
                });

                delete regions[leftRegion];
            }
            else if (left && left.plantType === area[i][j].plantType) {
                area[i][j].region = left.region;
            }
            else if (top && top.plantType === area[i][j].plantType) {
                area[i][j].region = top.region;
            }
            else {
                currentRegion += 1;
                regions[currentRegion] = [];
                area[i][j].region = currentRegion;
            }
            regions[area[i][j].region].push(area[i][j]);
        }
    }
    return [area, regions];
}

const [area, regions] = populateRegions(area);
Object.keys(regions).length

11

In [280]:
// part 1 - What is the total price of fencing all regions on your map?

const calculatePerimeter = (plots: plot[], area: plot[][]) => {
    let perimeter = 0;
    for (let i = 0; i < plots.length; i++) {
        const plot = plots[i];
        [[-1, 0], [1, 0], [0, -1], [0, 1]].forEach(([x, y]) => {
            const neighbor = area[plot.x + x] ? area[plot.x + x][plot.y + y] : null;
            if (!neighbor || neighbor.region !== plots[i].region) {
                perimeter += 1;
            }
        });
    }
    return perimeter;
}

const calculateFencePrice = (regions: { [string]: plot[] }, area: plot[][]) => {
    let price = 0
    Object.keys(regions).forEach((region) => {
        const plots = regions[region];
        const plotArea = plots.length;
        const perimeter = calculatePerimeter(plots, area);
        price += plotArea * perimeter;
    });
    return price;
}
calculateFencePrice(regions, area)

1930

In [281]:
// Part 2 - What is the new total price of fencing all regions on your map with bulk discount ?

const calculateSides = (plots: plot[], area: plot[][]) => {
    if (plots.length === 1) return 4;
    let sides = 0;
    for (let i = 0; i < plots.length; i += 1) {
        const current = plots[i];
        const leftTopDiagonalNeighbor = area[current.x - 1] ? area[current.x - 1][current.y - 1]?.region === current.region : false;
        const rightTopDiagonalNeighbor = area[current.x - 1] ? area[current.x - 1][current.y + 1]?.region === current.region : false;
        const leftBottomDiagonalNeighbor = area[current.x + 1] ? area[current.x + 1][current.y - 1]?.region === current.region : false;
        const rightBottomDiagonalNeighbor = area[current.x + 1] ? area[current.x + 1][current.y + 1]?.region === current.region : false;

        const top = area[current.x - 1] ? area[current.x - 1][current.y]?.region === current.region : false;
        const bottom = area[current.x + 1] ? area[current.x + 1][current.y]?.region === current.region : false;
        const left = area[current.x][current.y - 1]?.region === current.region;
        const right = area[current.x][current.y + 1]?.region === current.region;

        // external corners
        if(!top && !right) sides += 1;
        if(!right && !bottom) sides += 1;
        if(!bottom && !left) sides += 1;
        if(!left && !top) sides += 1;

        // internal corners
        if(top && left && !leftTopDiagonalNeighbor) sides += 1;
        if(top && right && !rightTopDiagonalNeighbor) sides += 1;
        if(bottom && left && !leftBottomDiagonalNeighbor) sides += 1;
        if(bottom && right && !rightBottomDiagonalNeighbor) sides += 1;
    }
    return sides;
}

const calculateFencePriceV2 = (regions: { [string]: plot[] }, area: plot[][]) => {
    let price = 0
    Object.keys(regions).forEach((region) => {
        const plots = regions[region];
        const plotArea = plots.length;
        const sides = calculateSides(plots, area);
        price += plotArea * sides;
    });
    return price;
}
calculateFencePriceV2(regions, area)

1206